#Suumoスクレイピングで情報取得

東京都千代田区のエリアにおいて下記の必要最低限を含む情報を取得し、データ抽出し格納、csvファイルで保存する。

◇物件情報

・物件名

・住所


◇部屋情報

・間取り

・家賃

・階＃＃

In [1]:
#ライブラリのインポート
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

#アクセス先のURLの設定
REQUEST_URL = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page={}'

#空のリストを作成
d_list = []

#複数ページをforループにて取得（33ページ目まで）
for i in range(1,34):
    print("d_listの大きさ",len(d_list))

    TARGET_URL = REQUEST_URL.format(i)

    print(TARGET_URL)

    #requestにより設定したURLにアクセス
    res = requests.get(TARGET_URL)

    #1秒間のsleepをはさむ
    sleep(1)

    #取得した情報をbeautiful Soupで解析
    soup = BeautifulSoup(res.text,"html.parser")

        #すべての物件情報を取得
    contents = soup.find_all('div',class_="cassetteitem")

    #各物件情報をforループで取得
    for content in contents:
        #物件情報と部屋情報をそれぞれ取得
        detail = content.find('div',class_="cassetteitem-detail")
        item = content.find('div',class_="cassetteitem-item")

        #物件情報から物件名、住所を取得
        title = detail.find('div',class_='cassetteitem_content-title').text
        address = detail.find('li',class_='cassetteitem_detail-col1').text

        #部屋情報から間取り、家賃、階を取得するための情報を取得
        tr_tags = item.find_all("tr", class_="js-cassette_link")

        #部屋情報は複数あり、各部屋情報を取得
        for tr_tag in tr_tags:
            #各部屋情報から取得したい情報をそれぞれの変数に格納
            floor,price,first_fee,capacity = tr_tag.find_all("td")[2:6]

            #priceの中身を賃料(fee)、管理費(management_fee)でそれぞれ分けて格納
            fee, management_fee = price.find_all("li")

            #first_feeの中身を敷金(deposit)、礼金(gratuity)でそれぞれ分けて格納
            deposit, gratuity = first_fee.find_all("li")

            #capacityの中身を間取り(madori)、専有面積(menseki)でそれぞれ分けて格納
            madori, menseki = capacity.find_all("li")

            #変数dにこれまで取得した9項目を格納する
            d = {
                "title":title,
                "address":address,
                "floor":floor.text,
                "fee":fee.text,
                "management_fee":management_fee.text,
                "deposit":deposit.text,
                "gratuity":gratuity.text,
                "madori":madori.text,
                "menseki":menseki.text
            }

            #取得した辞書をd_listに格納する
            d_list.append(d)

d_listの大きさ 0
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=1
d_listの大きさ 136
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=2
d_listの大きさ 248
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=3
d_listの大きさ 426
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=4
d_listの大きさ 546
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr

In [2]:
#取得した情報をデータフレームに格納
df = pd.DataFrame(d_list)

In [3]:
#dfの大きさを確認する
df.shape

(2371, 9)

In [4]:
# 各カラムのユニークな値を確認
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in column {column}:\n{unique_values}\n")

Unique values in column title:
['ロイジェント九段' 'KWレジデンス麹町' 'ピークス神田' '東京メトロ銀座線 神田駅 12階建 築3年' 'パークリュクス神田'
 'Ｌｅ\u3000Ｎｉｄ\u3000ＫＡＮＤＡ（ルニ神田）' 'シュトラーレ四番町' '市ヶ谷スクエアレジデンス' 'ハーモニーレジデンス秋葉原'
 'ＪＲ山手線 神田駅 14階建 築7年' 'ＪＲ山手線 神田駅 13階建 築19年' 'ＪＲ山手線 神田駅 13階建 築18年'
 '東京メトロ日比谷線 小伝馬町駅 12階建 築10年' 'アビタシオン創' 'グラン，フォークス神田イーストタワー'
 'レ・ジェイドクロス千代田神保町' 'S-RESIDENCE神田avance' 'ＪＲ中央線 神田駅 9階建 築8年' 'パティオ神田'
 '東京メトロ半蔵門線 神保町駅 14階建 新築' 'Nステージ秋葉原' 'アルティオドマーニ【Artio Domani】'
 '東京メトロ半蔵門線 九段下駅 14階建 築12年' 'N35 EAST' 'ガリシア九段下ライズ' 'ＪＲ中央線 四ツ谷駅 14階建 築2年'
 'プレミアステージ内神田' 'ＪＲ中央線 水道橋駅 10階建 築9年' 'ＪＲ山手線 秋葉原駅 15階建 築17年' 'フォレシティ神田須田町'
 'エクレール平河町' 'メインステージ神田須田町' '東京メトロ銀座線 神田駅 14階建 築8年' '都営新宿線 岩本町駅 8階建 築21年'
 'エスコート神田岩本町' '都営新宿線 岩本町駅 12階建 築20年' 'CITY CURRENT大手町\u3000シティカレント大手町'
 'マーシュフィールド四番町' 'コンフォリア秋葉原EAST' 'コンフォリア秋葉原イースト' 'インペリアル御茶の水'
 '東京メトロ有楽町線 市ケ谷駅 地下1地上13階建 築26年' '東海西神田マンション' 'パークハウス三番町' 'プライムアーバン番町'
 'グランヴィスタ神田練塀町' '東京メトロ半蔵門線 半蔵門駅 18階建 築5年' 'ＪＲ総武線 市ケ谷駅 9階建 築42年'
 '東京メトロ半蔵門線 半蔵門駅 地下1地上14階建 築16年' 'コスモグラシア内神田' 'ディームス大手町ノース'
 'ＪＲ総武線 市ケ谷駅 地下1地上

In [5]:
#floorデータの不要な文字を削除
df["floor"] = df["floor"].str.replace("\r\n\t\t\t\t\t\t\t\t\t\t\t","")

#feeを数値型に変換し、fee /yen列を追加
df["fee /yen"] = (df["fee"].str.replace("万円","").astype(float)*10000).astype(int)

#management_feeを数値型に変換し、management_fee /yen列を追加
df["management_fee /yen"] = (df["management_fee"].str.replace("円", "").replace("-", "0")).astype(float).astype(int)

#depositを数値型に変換し、deposit /yen列を追加
df["deposit /yen"] = (df["deposit"].str.replace("万円","").replace("-", "0").astype(float)*10000).astype(int)

#gratuityを数値型に変換し、gratuity /yen列を追加
df["gratuity /yen"] = (df["gratuity"].str.replace("万円","").replace("-", "0").astype(float)*10000).astype(int)

#mensekiのデータを数値に変換し、gratuity /m2列にデータを格納し追加
df["menseki /m2"] = df["menseki"].str.replace("m2","").astype(float)

In [6]:
#データクレンジングの確認
df.head()

,title,address,floor,fee,management_fee,deposit,gratuity,madori,menseki,fee /yen,management_fee /yen,deposit /yen,gratuity /yen,menseki /m2
0,ロイジェント九段,東京都千代田区九段北１,2階,15.1万円,-,15.1万円,-,1K,30.4m2,151000,0,151000,0,30.40
1,ロイジェント九段,東京都千代田区九段北１,5階,15.2万円,-,15.2万円,-,1K,29.43m2,152000,0,152000,0,29.43
2,ロイジェント九段,東京都千代田区九段北１,10階,29.2万円,-,29.2万円,29.2万円,2LDK,61.33m2,292000,0,292000,292000,61.33
3,KWレジデンス麹町,東京都千代田区麹町３,6階,16.2万円,15000円,16.2万円,16.2万円,1K,33.1m2,162000,15000,162000,162000,33.10
4,KWレジデンス麹町,東京都千代田区麹町３,7階,16.2万円,15000円,16.2万円,16.2万円,1K,33.1m2,162000,15000,162000,162000,33.10


In [7]:
#データクレンジングの確認
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2371 entries, 0 to 2370
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                2371 non-null   object 
 1   address              2371 non-null   object 
 2   floor                2371 non-null   object 
 3   fee                  2371 non-null   object 
 4   management_fee       2371 non-null   object 
 5   deposit              2371 non-null   object 
 6   gratuity             2371 non-null   object 
 7   madori               2371 non-null   object 
 8   menseki              2371 non-null   object 
 9   fee /yen             2371 non-null   int32  
 10  management_fee /yen  2371 non-null   int32  
 11  deposit /yen         2371 non-null   int32  
 12  gratuity /yen        2371 non-null   int32  
 13  menseki /m2          2371 non-null   float64
dtypes: float64(1), int32(4), object(9)
memory usage: 222.4+ KB


In [8]:
#階層、間取り、賃料、管理費、専有面積で重複している行を削除
df = df.drop_duplicates(subset=["floor", "madori", "fee /yen", "management_fee /yen", "menseki /m2" ], keep="first")

In [9]:
# 不要な複数のカラムを削除
columns_to_drop = ["fee", "management_fee", "deposit", "gratuity", "menseki"]
df = df.drop(columns=columns_to_drop)

In [11]:
#重複、不要なカラム削除の確認
df.head()

,title,address,floor,madori,fee /yen,management_fee /yen,deposit /yen,gratuity /yen,menseki /m2
0,ロイジェント九段,東京都千代田区九段北１,2階,1K,151000,0,151000,0,30.40
1,ロイジェント九段,東京都千代田区九段北１,5階,1K,152000,0,152000,0,29.43
2,ロイジェント九段,東京都千代田区九段北１,10階,2LDK,292000,0,292000,292000,61.33
3,KWレジデンス麹町,東京都千代田区麹町３,6階,1K,162000,15000,162000,162000,33.10
4,KWレジデンス麹町,東京都千代田区麹町３,7階,1K,162000,15000,162000,162000,33.10


In [12]:
#重複、不要なカラム削除の確認
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1367 entries, 0 to 2370
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                1367 non-null   object 
 1   address              1367 non-null   object 
 2   floor                1367 non-null   object 
 3   madori               1367 non-null   object 
 4   fee /yen             1367 non-null   int32  
 5   management_fee /yen  1367 non-null   int32  
 6   deposit /yen         1367 non-null   int32  
 7   gratuity /yen        1367 non-null   int32  
 8   menseki /m2          1367 non-null   float64
dtypes: float64(1), int32(4), object(4)
memory usage: 85.4+ KB


In [13]:
#csvに変換して保存
df.to_csv("SUUMO_scraping_ver2.csv",index=None, encoding="utf-8-sig")